Neuronale Netze Tests
===
Die in dem XOR Neural Net Notebook erstellten Netze wurden in verschiedenen Klassen und Funktionen zusammengefasst. Deswegen wird diese Implementierung anhand des XOR Beispiel und des Iris Datensatz ausprobiert. Verwendet wird in den jeweiligen Architekturen die bereits getestete Idee der Quadratischen Funktion als Aktivierungsfunktion zusammen mit einer Sigmoid Funktion in der Ausgabenschicht.

In [27]:
from util.heNet import Network, FullyConnectedLayer, ActivationLayer
from util.heNet import square, square_prime, sigmoid, sigmoid_prime, binary_cross_entropy, binary_cross_entropy_prime
import numpy as np
import tenseal as ts

In [2]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

In [3]:
net = Network()
net.add(FullyConnectedLayer(2, 2))
net.add(ActivationLayer(square, square_prime))
net.add(FullyConnectedLayer(2, 1))
net.add(ActivationLayer(sigmoid, sigmoid_prime))

net.use(binary_cross_entropy, binary_cross_entropy_prime)

In [4]:
net.fit(x_train, y_train, epochs=100, learning_rate=1, minibatch=4)

out = net.predict(x_train)
print(out)

[array([[0.06464641]]), array([[0.9856887]]), array([[0.98586713]]), array([[0.06312294]])]


In diesem Test wurde die gleiche Architektur verwendet und es erzeugt die korrekten Ausgaben.

Iris Datensatz
---
Als nächster Test wird der Datensatz aus der Linear Regression verwendet. In diesem Fall wird aber die Art der Blumen anhand der Merkmalen vorhergesagt.

In [28]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

def transform(data):
    return np.array([[row] for row in data])

X_train = transform(X_train)
X_test = transform(X_test)
y_train = transform(y_train)
y_test = transform(y_test)

Die verwendete Architektur für die Iris Aufgabe besteht wie für XOR aus der gleichen Anzahl an Schichten, aber hat eine andere Dimension, da es unterschiedlich viele Features gibt.

In [56]:
net = Network(debug=None)
net.add(FullyConnectedLayer(4, 4))
net.add(ActivationLayer(square, square_prime))
net.add(FullyConnectedLayer(4, 3))
net.add(ActivationLayer(square, square_prime))

net.use(binary_cross_entropy, binary_cross_entropy_prime)
net.fit(X_train, y_train, epochs=10, learning_rate=0.002, minibatch=8)

Das trainierte Modell kann die Klassen der Blumen des Testdatensatz nun vorhersagen.

In [57]:
outHE = net.predict(X_test)
correct = 0
for idx,_ in enumerate(outHE):
    if np.argmax(np.squeeze(outHE[idx])) == np.argmax(np.squeeze(y_test[idx])):
        correct = correct + 1
print(f"Im Testdatensatz wurden {correct} von {len(outHE)} richtig klassifiziert --> {round(correct/len(outHE),2)}")

Im Testdatensatz wurden 29 von 30 richtig klassifiziert --> 0.97


Das HE freundliche Modell hat fast alle der unverschlüsselten Daten korrekt klassifiziert.

Zum Vergleich wird noch ein "normales" Netz trainiert, das aus der gleichen Anzahl an Schichten besteht aber anstatt der quadratischen Aktivierungsfunktion die Sigmoid Aktivierungsfunktion verwendet in der Versteckten Schicht.

In [5]:
net = Network(debug=None)
net.add(FullyConnectedLayer(4, 4))
net.add(ActivationLayer(sigmoid, sigmoid_prime))
net.add(FullyConnectedLayer(4, 3))
net.add(ActivationLayer(sigmoid, sigmoid_prime))

net.use(binary_cross_entropy, binary_cross_entropy_prime)
net.fit(X_train, y_train, epochs=10, learning_rate=0.1)

In [6]:
outN = net.predict(X_test)
correct = 0
for idx,_ in enumerate(outN):
    if np.argmax(np.squeeze(outN[idx])) == np.argmax(np.squeeze(y_test[idx])):
        correct = correct + 1
print(f"Im Testdatensatz wurden {correct} von {len(outN)} richtig klassifiziert --> {round(correct/len(outN),2)}")

Im Testdatensatz wurden 30 von 30 richtig klassifiziert --> 1.0


Die Genauigkeit beider Modelle ist ähnlich und weißt darauf hin, dass das HE freundliche Modell nicht schlechter ist als die übliche Architektur.